# PPI Network Retrieval and Analysis

In [2]:
import requests
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
scamps = ['SCAMP1', 'SCAMP2', 'SCAMP3', 'SCAMP4', 'SCAMP5']
interactor_ids = []

In [3]:
for protein in scamps:
    response = requests.get(f"https://string-db.org/api/json/network?identifier={protein}&species=9606")
    data = response.json()
    for interaction in data:
        interactor_ids.append((interaction['preferredName_A'], interaction['preferredName_B'], interaction['score']))

In [ ]:
print(f"Number of PPI's pulled from STRING DB: {len(interactor_ids)}")

In [ ]:
ppi_table = pd.DataFrame(interactor_ids, columns=('Interactor_A', 'Interactor_B', 'Confidence Score'))
ppi_table

In [6]:
ppi_graph = nx.Graph()
for node_a, node_b, score in interactor_ids:
    ppi_graph.add_edge(node_a, node_b, weight = score)

In [ ]:
plt.figure(figsize=(40,40))
nx.draw(ppi_graph, with_labels = True, font_size = 20)

Reflects the likelihood of interaction *in vivo* 

In [ ]:
from networkx.algorithms.community import greedy_modularity_communities

clusters = list(greedy_modularity_communities(ppi_graph))
for i, cluster in enumerate(clusters):
    print(f"Cluster {i+1}: {sorted(cluster)}")

In [ ]:
import os

f_dir = 'PPI_cluster_files/'
os.mkdir(f_dir)

for i, cluster in enumerate(clusters):
    with open(f"{f_dir}/cluster_{i+1}.txt", "w") as file:
        for protein in cluster:
            file.write(protein + "\n")

# Gene set Enrichment Analysis

## Cluster 1

In [11]:
import gseapy as gp

In [12]:
with open(f"{f_dir}/cluster_1.txt", 'r') as clus1_f:
    c1 = [protein.replace("\n", '') for protein in clus1_f.readlines()]

enr = gp.enrichr(gene_list=c1, gene_sets='KEGG_2021_Human', organism='Human')

c1_gsea_results = enr.results

In [ ]:
c1_gsea_results

## Cluster 2

In [ ]:
with open(f"{f_dir}/cluster_2.txt", 'r') as clus2_f:
    c2 = [protein.replace("\n", '') for protein in clus2_f.readlines()]

enr2 = gp.enrichr(gene_list=c2, gene_sets='KEGG_2021_Human', organism='Human')

c2_gsea_results = enr2.results    
c2_gsea_results

## Cluster 3

In [ ]:
with open(f"{f_dir}/cluster_3.txt", 'r') as clus3_f:
    c3 = [protein.replace("\n", '') for protein in clus3_f.readlines()]

enr3 = gp.enrichr(gene_list=c3, gene_sets='KEGG_2021_Human', organism='Human')

c3_gsea_results = enr3.results
c3_gsea_results

## Cluster 4

In [ ]:
with open(f"{f_dir}/cluster_4.txt", 'r') as clus4_f:
    c4 = [protein.replace("\n", '') for protein in clus4_f.readlines()]

enr4 = gp.enrichr(gene_list=c4, gene_sets='KEGG_2021_Human', organism='Human')

c4_gsea_results = enr4.results
c4_gsea_results

In [ ]:
c1_gsea_results['Cluster'] = 1
c2_gsea_results['Cluster'] = 2
c3_gsea_results['Cluster'] = 3
c4_gsea_results['Cluster'] = 4

gsea_all = pd.concat([c1_gsea_results, c2_gsea_results, c3_gsea_results, c4_gsea_results])
gsea_all

In [18]:
gsea_all.to_csv('scamp_ppi_gsea.csv', index = False)

In [ ]:
gsea_toplot = pd.read_csv('scamp_ppi_gsea.csv')
signifigant = gsea_toplot[gsea_toplot['Adjusted P-value'] <= 0.10]
signifigant

In [ ]:
import numpy as np
import seaborn as sns

for cluster in signifigant['Cluster'].unique():
    cluster_data = signifigant[signifigant['Cluster'] == cluster]
    top_path = cluster_data.sort_values('Adjusted P-value')
    
    plt.figure(figsize=(8,6))
    sns.barplot(y = top_path['Term'], x = -np.log10(top_path['Adjusted P-value']), hue = top_path['Term'], palette='viridis')
    plt.title(f'Top Pathways for Cluster {cluster}')
    plt.xlabel('-log10(Adjusted P-value)')
    plt.ylabel('Pathway Term')
    plt.tight_layout()
    plt.show()

# Using DAVID Knowledgebase to Functionally Annotate Interaction Clusters

I am using the DAVID knowledgebase (https://davidbioinformatics.nih.gov/) to functionally annotate the PPI clusters derived from network analysis. I am retrieving enriched Gene Ontology terms, InterPro Protein Domain Terms, and KEGG and Reactome Pathway terms to provide more semantics to the network analysis and hopefully distinguish the functional differences between the 4 different interaction networks. When reviewing this notebook, I highly suggest you go to the DAVID knowledgebase itself to get more indepth analysis regarding the data pulled from there. Especially when it comes to examining the KEGG and Reactome Pathways, as they can provide graphical schematics that depict enriched/interested genes in their respective pathways.[^1]

[^1]: Note - trying to figure out how to get programatic access through API but having trouble communicating with it

In [42]:
# ATTEMPT AT API ACCESS
# session = requests.Session()

# cex = '192683,115827,64837,6857,9900,10228,6810,54913,321,60490,1198'
# parameters1 = {
#     "type": "ENTREZ_GENE_ID",
#     "ids": cex,
#     "tool": "chartReport",
#     "annot": "GOTERM_BP_DIRECT,GOTERM_CC_DIRECT,GOTERM_MF_DIRECT,INTERPRO,KEGG_PATHWAY"
# }
# david_url1 = f'https://davidbioinformatics.nih.gov/api.jsp?'

# response1 = session.post(david_url1, data=parameters1)

# response1.text

### Cluster 1 DAVID Results

In [43]:
c1_BP = pd.read_csv('DAVID_results/cluster1_results/cluster1_GOTERM_BP_Direct.txt', sep = '\t')
c1_CC = pd.read_csv('DAVID_results/cluster1_results/cluster1_GOTERM_CC_Direct.txt', sep = '\t')
c1_MF = pd.read_csv('DAVID_results/cluster1_results/cluster1_GOTERM_MF_Direct.txt', sep='\t')
c1_IP = pd.read_csv('DAVID_results/cluster1_results/cluster1_InterProDomains.txt', sep='\t')
c1_KP = pd.read_csv('DAVID_results/cluster1_results/cluster1_KEGGPath.txt', sep='\t')
c1_RP = pd.read_csv('DAVID_results/cluster1_results/cluster1_Reactome.txt', sep='\t')
c1_david = pd.concat([c1_BP, c1_CC, c1_MF, c1_IP, c1_KP, c1_RP], axis = 0, ignore_index = True)
c1_david['Cluster'] = 1
c1_david = c1_david.drop('Category    ', axis = 1)
c1_david.loc[11:34, 'Category'] = 'GOTERM_CC_DIRECT'
c1_david

,Category,Term,Count,%,PValue,Genes,List Total,Pop Hits,Pop Total,Fold Enrichment,Bonferroni,Benjamini,FDR,Cluster
0,GOTERM_BP_DIRECT,GO:0006892~post-Golgi vesicle-mediated transport,4,25.00,1.661990e-07,"SNAP23, EXOC5, SCAMP1, SCAMP2",15,16,19416,323.600000,0.000018,0.000018,0.000017,1
1,GOTERM_BP_DIRECT,GO:0015031~protein transport,7,43.75,4.375149e-07,"VAMP8, JAGN1, SNAP23, ITSN1, EXOC5, SCAMP1, SC...",15,460,19416,19.697391,0.000046,0.000023,0.000022,1
2,GOTERM_BP_DIRECT,GO:0006887~exocytosis,5,31.25,1.368827e-06,"JAGN1, SNAP23, ITSN1, EXOC5, SCAMP1",15,121,19416,53.487603,0.000145,0.000048,0.000047,1
3,GOTERM_BP_DIRECT,GO:0006897~endocytosis,5,31.25,7.734581e-06,"ITSN2, SYNRG, ITSN1, SCAMP1, RAB5A",15,187,19416,34.609626,0.000820,0.000205,0.000197,1
4,GOTERM_BP_DIRECT,GO:0016197~endosomal transport,3,18.75,1.070333e-03,"ITSN2, ITSN1, ARHGAP1",15,68,19416,57.105882,0.107310,0.022691,0.021835,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,REACTOME_PATHWAY,R-HSA-199992~trans-Golgi Network Vesicle Budding,2,12.50,5.664282e-02,"VAMP8, SNAP23",10,72,11153,30.980556,0.991616,0.221177,0.194204,1
76,REACTOME_PATHWAY,R-HSA-1236974~ER-Phagosome pathway,2,12.50,6.046827e-02,"VAMP8, SNAP23",10,77,11153,28.968831,0.993992,0.225382,0.197896,1
77,REACTOME_PATHWAY,R-HSA-1236975~Antigen processing-Cross present...,2,12.50,7.186202e-02,"VAMP8, SNAP23",10,92,11153,24.245652,0.997791,0.250687,0.220116,1
78,REACTOME_PATHWAY,R-HSA-9013423~RAC3 GTPase cycle,2,12.50,7.337187e-02,"SNAP23, ARHGAP1",10,94,11153,23.729787,0.998067,0.250687,0.220116,1


### Cluster 2 DAVID Results

In [44]:
c2_BP = pd.read_csv('DAVID_results/cluster2_results/cluster2_GOTERM_BP_Direct.txt', sep = '\t').reset_index(drop=True)
c2_CC = pd.read_csv('DAVID_results/cluster2_results/cluster2_GOTERM_CC_Direct.txt', sep = '\t').reset_index(drop=True)
c2_MF = pd.read_csv('DAVID_results/cluster2_results/cluster2_GOTERM_MF_direct.txt', sep='\t').reset_index(drop=True)
c2_IP = pd.read_csv('DAVID_results/cluster2_results/cluster2_InterProDomains.txt', sep='\t').reset_index(drop=True)
c2_KP = pd.read_csv('DAVID_results/cluster2_results/cluster2_KEGGPath.txt', sep='\t').reset_index(drop=True)
c2_RP = pd.read_csv('DAVID_results/cluster2_results/cluster2_Reactome.txt', sep='\t').reset_index(drop=True)

c2_david = pd.concat([c2_BP, c2_CC, c2_MF, c2_IP, c2_KP, c2_RP], ignore_index = True)
c2_david['Cluster'] = 2

c2_david

,Category,Term,Count,%,PValue,Genes,List Total,Pop Hits,Pop Total,Fold Enrichment,Bonferroni,Benjamini,FDR,Cluster
0,GOTERM_BP_DIRECT,GO:0048278~vesicle docking,3,27.272727,4.987666e-05,"SYT1, STX6, STX4",11,21,19416,252.155844,0.003982,0.003132,0.003015,2
1,GOTERM_BP_DIRECT,GO:0006906~vesicle fusion,3,27.272727,1.030608e-04,"SYT1, STX6, STX4",11,30,19416,176.509091,0.008211,0.003132,0.003015,2
2,GOTERM_BP_DIRECT,GO:0017157~regulation of exocytosis,3,27.272727,1.174484e-04,"RAB3C, SYT1, STX4",11,32,19416,165.477273,0.009352,0.003132,0.003015,2
3,GOTERM_BP_DIRECT,GO:0006887~exocytosis,3,27.272727,1.677564e-03,"SCAMP5, RAB3C, STX4",11,121,19416,43.762585,0.125688,0.033551,0.032293,2
4,GOTERM_BP_DIRECT,GO:0007268~chemical synaptic transmission,3,27.272727,5.955495e-03,"SYT1, SV2A, APBA2",11,231,19416,22.923259,0.379893,0.095288,0.091715,2
5,GOTERM_BP_DIRECT,GO:0015031~protein transport,3,27.272727,2.222245e-02,"SCAMP5, RAB3C, APBA2",11,460,19416,11.511462,0.834345,0.267956,0.257908,2
6,GOTERM_BP_DIRECT,GO:0006836~neurotransmitter transport,2,18.181818,2.344618e-02,"SV2A, STX4",11,46,19416,76.743083,0.850137,0.267956,0.257908,2
7,GOTERM_CC_DIRECT,GO:0030672~synaptic vesicle membrane,5,45.454545,3.763314e-07,"SCAMP5, RAB3C, SYT1, SV2A, STX6",11,137,20666,68.566689,0.000031,0.000031,0.000031,2
8,GOTERM_CC_DIRECT,GO:0008021~synaptic vesicle,4,36.363636,2.826073e-05,"RAB3C, SYT1, SV2A, STX4",11,130,20666,57.806993,0.002343,0.001173,0.001159,2
9,GOTERM_CC_DIRECT,GO:0005886~plasma membrane,8,72.727273,5.332445e-03,"SCAMP5, RAB3C, KLC2, SYT1, SV2A, STX6, STX4, A...",11,5534,20666,2.715905,0.358392,0.147531,0.145754,2


### Cluster 3 DAVID Results

In [45]:
c3_CC = pd.read_csv('DAVID_results/cluster3_results/cluster3_GOTERM_CC_Direct.txt', sep = '\t').reset_index(drop=True)
c3_KP = pd.read_csv('DAVID_results/cluster3_results/cluster3_KEGGPath.txt', sep='\t').reset_index(drop=True)

c3_david = pd.concat([c3_CC, c3_KP], ignore_index = True)
c3_david['Cluster'] = 3

c3_david

,Category,Term,Count,%,PValue,Genes,List Total,Pop Hits,Pop Total,Fold Enrichment,Bonferroni,Benjamini,FDR,Cluster
0,GOTERM_CC_DIRECT,GO:0005789~endoplasmic reticulum membrane,3,30.0,0.072615,"POMT2, REEP5, XXYLT1",9,1181,20666,5.832910,0.836226,1.000000,1.000000,3
1,KEGG_PATHWAY,hsa00514:Other types of O-glycan biosynthesis,2,20.0,0.015823,"POMT2, XXYLT1",4,47,8865,94.308511,0.076650,0.079114,0.079114,3


### Cluster 4 DAVID Results

In [46]:
c4_BP = pd.read_csv('DAVID_results/cluster4_results/cluster4_GOTERM_BP_Direct.txt', sep = '\t').reset_index(drop=True)
c4_CC = pd.read_csv('DAVID_results/cluster4_results/cluster4_GOTERM_CC_Direct.txt', sep = '\t').reset_index(drop=True)
c4_MF = pd.read_csv('DAVID_results/cluster4_results/cluster4_GOTERM_MF_direct.txt', sep='\t').reset_index(drop=True)
c4_KP = pd.read_csv('DAVID_results/cluster4_results/cluster4_KEGGPath.txt', sep='\t').reset_index(drop=True)
c4_RP = pd.read_csv('DAVID_results/cluster4_results/cluster4_Reactome.txt', sep='\t').reset_index(drop=True)

c4_david = pd.concat([c4_BP, c4_CC, c4_MF, c4_KP, c4_RP], ignore_index = True)
c4_david['Cluster'] = 4

c4_david

,Category,Term,Count,%,PValue,Genes,List Total,Pop Hits,Pop Total,Fold Enrichment,Bonferroni,Benjamini,FDR,Cluster
0,GOTERM_BP_DIRECT,GO:0043405~regulation of MAP kinase activity,2,25.0,0.002881,"TSG101, HGS",8,8,19416,606.750000,0.180518,0.140618,0.132466,4
1,GOTERM_BP_DIRECT,GO:1903543~positive regulation of exosomal sec...,2,25.0,0.005396,"TSG101, HGS",8,15,19416,323.600000,0.311574,0.140618,0.132466,4
2,GOTERM_BP_DIRECT,GO:0043328~protein transport to vacuole involv...,2,25.0,0.006114,"TSG101, HGS",8,17,19416,285.529412,0.345019,0.140618,0.132466,4
3,GOTERM_BP_DIRECT,GO:0015031~protein transport,3,37.5,0.010869,"TSG101, HGS, SCAMP3",8,460,19416,15.828261,0.529564,0.158448,0.149263,4
4,GOTERM_BP_DIRECT,GO:0036258~multivesicular body assembly,2,25.0,0.011482,"TSG101, HGS",8,32,19416,151.687500,0.549242,0.158448,0.149263,4
5,GOTERM_BP_DIRECT,GO:0090148~membrane fission,2,25.0,0.014691,"TSG101, HGS",8,41,19416,118.390244,0.639826,0.168942,0.159148,4
6,GOTERM_BP_DIRECT,GO:0032456~endocytic recycling,2,25.0,0.024263,"HGS, RAB11B",8,68,19416,71.382353,0.816369,0.221446,0.208609,4
7,GOTERM_BP_DIRECT,GO:0016236~macroautophagy,2,25.0,0.025675,"TSG101, HGS",8,72,19416,67.416667,0.833824,0.221446,0.208609,4
8,GOTERM_BP_DIRECT,GO:0016525~negative regulation of angiogenesis,2,25.0,0.047329,"NAXE, HGS",8,134,19416,36.223881,0.964758,0.362856,0.341821,4
9,GOTERM_CC_DIRECT,GO:0070062~extracellular exosome,5,62.5,0.003689,"TSG101, NAXE, HGS, SCAMP3, RAB11B",8,2242,20666,5.761039,0.150094,0.162329,0.162329,4


In [48]:
all_david = pd.concat([c1_david, c2_david, c3_david, c4_david], ignore_index=True)
all_david

,Category,Term,Count,%,PValue,Genes,List Total,Pop Hits,Pop Total,Fold Enrichment,Bonferroni,Benjamini,FDR,Cluster
0,GOTERM_BP_DIRECT,GO:0006892~post-Golgi vesicle-mediated transport,4,25.00,1.661990e-07,"SNAP23, EXOC5, SCAMP1, SCAMP2",15,16,19416,323.600000,0.000018,0.000018,0.000017,1
1,GOTERM_BP_DIRECT,GO:0015031~protein transport,7,43.75,4.375149e-07,"VAMP8, JAGN1, SNAP23, ITSN1, EXOC5, SCAMP1, SC...",15,460,19416,19.697391,0.000046,0.000023,0.000022,1
2,GOTERM_BP_DIRECT,GO:0006887~exocytosis,5,31.25,1.368827e-06,"JAGN1, SNAP23, ITSN1, EXOC5, SCAMP1",15,121,19416,53.487603,0.000145,0.000048,0.000047,1
3,GOTERM_BP_DIRECT,GO:0006897~endocytosis,5,31.25,7.734581e-06,"ITSN2, SYNRG, ITSN1, SCAMP1, RAB5A",15,187,19416,34.609626,0.000820,0.000205,0.000197,1
4,GOTERM_BP_DIRECT,GO:0016197~endosomal transport,3,18.75,1.070333e-03,"ITSN2, ITSN1, ARHGAP1",15,68,19416,57.105882,0.107310,0.022691,0.021835,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,KEGG_PATHWAY,hsa04144:Endocytosis,3,37.50,2.369422e-03,"TSG101, HGS, RAB11B",4,252,8865,26.383929,0.018799,0.018955,0.018955,4
131,KEGG_PATHWAY,hsa03250:Viral life cycle - HIV-1,2,25.00,2.150463e-02,"TSG101, HGS",4,64,8865,69.257812,0.159631,0.086019,0.086019,4
132,REACTOME_PATHWAY,R-HSA-917729~Endosomal Sorting Complex Require...,2,25.00,1.142896e-02,"TSG101, HGS",5,32,11153,139.412500,0.486598,0.389822,0.389822,4
133,REACTOME_PATHWAY,R-HSA-199991~Membrane Trafficking,3,37.50,1.798223e-02,"TSG101, HGS, RAB11B",5,635,11153,10.538268,0.650922,0.389822,0.389822,4
